In [1]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import and Logging setup

from flask import Flask, request, jsonify
from model import predict_anomaly
import csv
import os

#Defines the Flask app, exposes REST endpoints 
app = Flask(__name__)

#Logging setup log_dir
log_dir = "logs"
log_file = os.path.join(log_dir, "predictions_log.csv")
os.makedirs(log_dir, exist_ok=True)

if not os.path.exists(log_file):
    with open(log_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp",
            "machine_id",
            "temperature",
            "humidity",
            "sound",
            "anomaly_flag",
            "anomaly_score"
        ])

#Verify API is running
@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "running"})
    
#Validates IoT data, runs anomaly detection, log result, return prediction.
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()

    #Variefy sensor fields
    required_fields = ["timestamp", "machine_id", "temperature", "humidity", "sound"]
    for field in required_fields:
        if field not in data:
            return jsonify({"error": f"Missing field: {field}"}), 400

    anomaly_flag, anomaly_score = predict_anomaly(data)
    
    #Logging to csv 
    with open(log_file, mode="a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            data["timestamp"],
            data["machine_id"],
            data["temperature"],
            data["humidity"],
            data["sound"],
            anomaly_flag,
            anomaly_score
        ])

    return jsonify({
        "machine_id": data["machine_id"],
        "anomaly_flag": anomaly_flag,
        "anomaly_score": anomaly_score
    })

if __name__ == "__main__":
    app.run(debug=False)


Done
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
